# P with codon usage levels at different taxonomic levels

In [18]:
!pip install biopython

  Using cached biopython-1.78.tar.gz (16.9 MB)
  Created wheel for biopython: filename=biopython-1.78-cp39-cp39-win_amd64.whl size=2266490 sha256=4481654414e69170175174da5b5f2c86c7dde4d29714b3c710a21a65d109cef3
  Stored in directory: c:\users\yz\appdata\local\pip\cache\wheels\e1\ec\b2\4609a82fd96fdafc83df0238d69eea90c1b7b7036f59683e17
Successfully built biopython


In [16]:
!pip install pandas

## Data download process
The data is available from https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs. The Refseq_species.tsv contains all the codon frequencies of many species from NCBI. Firstly we try to download via the download link with wget.

In [15]:
import os
import wget
print(myfile)
data_folder = "./data"
filename = "codon_usage.tsv"
if not os.path.exists(data_folder):
    os.makedirs(data_folder)
filepath = os.path.join(data_folder,filename)
url = "https://hive.biochemistry.gwu.edu/dna.cgi?cmd=objFile&ids=586358&filename=Refseq_species.tsv&raw=1"
wget.download(url, filepath)
with open("data/codon_usage.tsv", "r") as f:
    print(f.readlines())

<Response [200]>
[]


However, the above approach did not work and returned an empty file. After examining the download process with the developers tool in Google Chrome, I realise it requires some headers for the GET request. I tried the below copying the headers in but it still did not work.

In [1]:
import requests
headers = {
"Host": "hive.biochemistry.gwu.edu",
"Connection": "keep-alive",
"sec-ch-ua": '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
"sec-ch-ua-mobile": "?0",
"Upgrade-Insecure-Requests": "1",
"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36",
"Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9",
"Sec-Fetch-Site": "same-origin",
"Sec-Fetch-Mode": "navigate",
"Sec-Fetch-User": "?1",
"Sec-Fetch-Dest": "document",
"Referer": "https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs",
"Accept-Encoding": "gzip, deflate, br",
"Accept-Language": "en-US,en;q=0.9",

"Range": "bytes=783482-783482",
"If-Range": "1578517013:86024007"}
myfile = requests.get("https://hive.biochemistry.gwu.edu/dna.cgi?cmd=objFile&ids=586358&filename=Refseq_species.tsv&raw=1", headers=headers)
print(myfile.content)

b''


Once again, it is empty. Hence, I resorted to using Selenium and navigating the browser. First, I downloaded the Chrome driver. Then, I used the browser developer tools to examine the webpage and copy the XPATH of the elements that have to be clicked. On normal navigation, it requires 2 clicks to initate the download of the file. 

In [2]:
from zipfile import ZipFile
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

#downloading and extracting the chrome drive
wget.download("https://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip")
file_name = "chromedriver_win32.zip"
with ZipFile(file_name, 'r') as zip:
    zip.extractall()

driver = webdriver.Chrome(executable_path=r'./chromedriver.exe')
driver.get("https://hive.biochemistry.gwu.edu/dna.cgi?cmd=tissue_codon_usage&id=586358&mode=cocoputs")
timeout = 60
try:
    element_present = EC.presence_of_element_located((By.XPATH, '//*[@id="results"]/li[3]/a'))
    WebDriverWait(driver, timeout).until(element_present)
    download_tab = driver.find_element_by_xpath("/html/body/form[2]/div/div/div[4]/div[1]/div[1]/div[2]/ul/li[3]/ul/li[3]/a")
    download_tab.click()
    print("Clicked the first button!")
except TimeoutException:
    print ("Timed out waiting for page to load")

try:
    element_present = EC.presence_of_element_located((By.XPATH, '/html/body/form[2]/div/div/div[7]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[18]/td[2]'))
    WebDriverWait(driver, timeout).until(element_present)
    download = driver.find_element_by_xpath('/html/body/form[2]/div/div/div[7]/div[1]/div[2]/div/div/div[2]/table/tbody/tr[18]/td[2]')
    download.click()
    print("Clicked the second tab!")
except TimeoutException:
    print ("Timed out waiting for page to load")

driver.close()

Clicked the first button!
Timed out waiting for page to load


However, this method ultimately did not work as the second table did not appear after clicking the first click. I was unable to make more progress due to my limited knowledge of HTML and JavaScript. Ultimately, I postulates that there are safeguards put in place to limit any automated downloading/conceal the download as they are an academic institute and may be afraid of request overload. I moved on by downloading the data manually (as of 13th of March). 

## Data exploration
Here, we look at how many entries are there in the file. 

In [143]:
import pandas as pd
df = pd.read_table("data/o586358-Refseq_species.tsv",index_col=False,low_memory=False)
print(df.shape)
df.head()

(191025, 76)


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107


In [144]:
print(list(df))
print(df.iloc[:, 0:6].nunique())
df.iloc[:,4].unique()

['Division', 'Assembly', 'Taxid', 'Species', 'Organelle', 'Translation Table', '# CDS', '# Codons', 'GC%', 'GC1%', 'GC2%', 'GC3%', 'TTT', 'TTC', 'TTA', 'TTG', 'CTT', 'CTC', 'CTA', 'CTG', 'ATT', 'ATC', 'ATA', 'ATG', 'GTT', 'GTC', 'GTA', 'GTG', 'TAT', 'TAC', 'TAA', 'TAG', 'CAT', 'CAC', 'CAA', 'CAG', 'AAT', 'AAC', 'AAA', 'AAG', 'GAT', 'GAC', 'GAA', 'GAG', 'TCT', 'TCC', 'TCA', 'TCG', 'CCT', 'CCC', 'CCA', 'CCG', 'ACT', 'ACC', 'ACA', 'ACG', 'GCT', 'GCC', 'GCA', 'GCG', 'TGT', 'TGC', 'TGA', 'TGG', 'CGT', 'CGC', 'CGA', 'CGG', 'AGT', 'AGC', 'AGA', 'AGG', 'GGT', 'GGC', 'GGA', 'GGG']
Division                  1
Assembly             177439
Taxid                 59670
Species               59060
Organelle                 4
Translation Table        11
dtype: int64


array(['genomic', 'mitochondrion', 'chloroplast', 'plastid'], dtype=object)

As we can see here, we have 191025 samples with 76 variables. A quick glance at the number of unique values show that the data requires cleaning. Firstly, we have less unique assemblies than number of rows, meaning that some of the assemblies are repeated or NaN. Even though the number of species and taxid differs, here it is acceptable as there might taxonomic IDs only goes up till species levels but there might be subspecies especially for microbes. We also have different type of DNA origins, here we focus on genomic DNA as those genes in mitochondrion, plastid or chloroplast might be shared between distantly related species and skew the differences. 

In [145]:
df[df.duplicated(['Assembly'])]

,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
658,refseq,GCF_000513435.1,1173487,Halorubrum sp. AJ67,genomic,11,3078,763055,64.68,67.31,...,8366,17661,3223,10213,1988,2032,7707,29947,8715,16798
660,refseq,GCF_000513435.1,1173487,Halorubrum sp. AJ67,genomic,11,3083,764041,64.69,67.32,...,8369,17683,3223,10222,1988,2032,7711,30002,8725,16820
719,refseq,NaN,1407055,Candidatus Nitrosotenuis uzonensis,genomic,11,1,200,55.83,60.50,...,1,0,1,1,2,2,0,2,5,0
720,refseq,GCF_000723185.1,1407055,Candidatus Nitrosotenuis uzonensis,genomic,11,1876,493394,42.86,49.86,...,2243,824,3560,5730,9304,5257,6294,10132,12660,3397
4956,refseq,NaN,1354719,Helicobacter pylori HP87P7,genomic,11,4,1064,37.53,40.04,...,11,8,10,22,21,21,9,10,14,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191020,refseq,NaN,1923763,Xingshan nematode virus 4,genomic,0,5,3630,38.21,43.91,...,49,13,60,29,72,21,40,19,85,41
191021,refseq,NaN,1923772,Xinzhou nematode virus 4,genomic,0,5,3652,39.91,45.29,...,39,11,71,30,73,12,38,21,99,34
191022,refseq,NaN,1415147,Xylella phage Salvo,genomic,11,72,17848,63.15,62.21,...,58,140,30,344,10,22,163,993,71,109
191023,refseq,NaN,1415148,Xylella phage Sano,genomic,11,77,17851,62.46,61.59,...,71,137,41,340,10,24,185,993,74,95


Interestingly, there are multiple entries with the same assembly ID but differing number of coding sequences (# CDS). This may be due to the different gene open reading fram predictions from different softwares. To simplify the data, we shall only keep the assembly with the largest number of CDS. Furhtermore, we shall remove those without assembly IDs, as they are just draft collections of contigs without proper curated annotations.  

In [146]:
df = df[df["Organelle"] == "genomic"] #keep genomic
df = df[df["Assembly"].notna()] #drop NaN
df = df.sort_values("# CDS", ascending=False).drop_duplicates("Assembly").sort_index()
print(df.shape)
df[df.duplicated(["Assembly"])]

(177439, 76)


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG


Looks like the data is cleaned! We shall check for any NAs just to be safe..

In [147]:
print(df.isnull().any().any())
df.head()

False


,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107


## Preparing data for training and testing
We will split the data up, using the codons as variables (total of 64) and using the taxonomy IDs as class labels. We will need to parse the taxonomy tree for the lineage of each taxonomy id. 

In [31]:
from Bio import Entrez
#entrez records has maximum length of 10000
total_species = len(df.iloc[:, 2].unique())
i=0
taxid = []
while i < total_species:
    if i == 50000:
        i_end = total_species
    else:
        i_end = i+10000
    handle = Entrez.efetch(db="Taxonomy", id=df.iloc[:, 2].unique()[i:i_end], retmode="xml")
    records = Entrez.read(handle)
    taxid.append(records)
    i+=10000

50398


In [148]:
tax_df = pd.DataFrame(0, index=range(total_species), columns=range(7))
major_taxonomy_levels = ["superkingdom", "phylum", "class", "order", "family", "genus", "Taxid"]
tax_df.columns = major_taxonomy_levels

In [152]:
i=0
for records in taxid:
    for record in records:
        tax_df.iloc[i,6] = int(record["TaxId"])
        for tax in record["LineageEx"]:
            if tax["Rank"] == "superkingdom":
                tax_df.iloc[i,0] = int(tax["TaxId"])
            elif tax["Rank"] == "phylum":
                tax_df.iloc[i,1] = int(tax["TaxId"])
            elif tax["Rank"] == "class":
                tax_df.iloc[i,2] = int(tax["TaxId"])
            elif tax["Rank"] == "order":
                tax_df.iloc[i,3] = int(tax["TaxId"])
            elif tax["Rank"] == "family":
                tax_df.iloc[i,4] = int(tax["TaxId"])
            elif tax["Rank"] == "genus":
                tax_df.iloc[i,5] = int(tax["TaxId"])
        i+=1
tax_df       

,superkingdom,phylum,class,order,family,genus,Taxid
0,2157,28890,183925,2158,2159,2160,2161
1,2157,28890,183925,2158,2159,2160,1204725
2,2157,28890,183925,2158,2159,2160,2162
3,2157,28890,183925,2158,2159,2172,420247
4,2157,28890,183925,2158,2159,2172,521002
...,...,...,...,...,...,...,...
50393,2759,7711,8292,8445,1277737,264009,1415580
50394,2759,7711,186623,7952,7953,75365,1608454
50395,2759,7711,186623,41712,31092,91732,1676925
50396,2759,7711,1338369,8459,8487,38771,1825980


In [150]:
print((tax_df == 0).sum(axis=0))
print(len(tax_df.iloc[:, 1].unique()))
from collections import Counter
Counter(tax_df["phylum"]).values()

superkingdom       0
phylum            71
class            785
order            424
family           837
genus           1077
Taxid              0
dtype: int64
88


dict_values([603, 91, 23, 33, 3, 1, 1, 71, 1, 21867, 8127, 4, 2663, 148, 615, 23, 13256, 116, 74, 183, 134, 43, 11, 32, 26, 547, 354, 83, 9, 132, 29, 50, 3, 1, 1, 5, 4, 3, 1, 15, 2, 6, 2, 2, 22, 1, 5, 1, 3, 1, 4, 1, 1, 2, 1, 3, 1, 1, 6, 4, 210, 4, 52, 8, 3, 9, 4, 4, 1, 9, 143, 1, 2, 309, 1, 1, 1, 1, 108, 8, 1, 2, 1, 7, 13, 4, 35, 1])

There appears to be some data with missing taxonomic annotations. Furthermore, some phyla has many examples while others only have a handful. We will only keep those with > 100 examples.

In [151]:
phylum_to_keep = [key for key,value in Counter(tax_df["phylum"]).items() if value > 100]
tax_df = tax_df.loc[tax_df["phylum"].isin(phylum_to_keep)]
taxid_to_keep = tax_df["Taxid"]
df = df.loc[df["Taxid"].isin(taxid_to_keep)]
df

,Division,Assembly,Taxid,Species,Organelle,Translation Table,# CDS,# Codons,GC%,GC1%,...,CGA,CGG,AGT,AGC,AGA,AGG,GGT,GGC,GGA,GGG
0,refseq,GCF_002287175.1,2161,Methanobacterium bryantii,genomic,11,3266,910953,35.52,45.52,...,2404,1515,12496,7687,14043,8418,18911,8417,26631,8789
1,refseq,GCF_000302455.1,1204725,Methanobacterium formicicum DSM 3637,genomic,11,2520,719599,40.88,49.69,...,2968,3068,10562,4882,8084,6214,17552,5296,21059,7069
2,refseq,GCF_000762265.1,2162,Methanobacterium formicicum,genomic,11,2376,678104,43.72,51.53,...,3067,5204,9335,4793,6499,5439,15945,5847,19130,7839
3,refseq,GCF_001458655.1,2162,Methanobacterium formicicum,genomic,11,2372,690178,43.50,51.30,...,3102,5182,9545,4797,6613,5472,16184,5887,19385,7934
4,refseq,GCF_000016525.1,420247,Methanobrevibacter smithii ATCC 35061,genomic,11,1757,557617,32.06,43.34,...,405,168,8234,3248,10191,2880,14664,2965,16981,2107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181848,refseq,GCF_901765095.1,1415580,Microcaecilia unicolor,genomic,0,35405,20605114,46.78,52.18,...,137037,136305,330959,359823,321754,242704,264062,274277,410074,241232
181850,refseq,GCF_001515605.1,1608454,Sinocyclocheilus anshuiensis,genomic,0,67753,41525357,50.84,54.62,...,266568,284411,622434,819038,615112,470561,550460,697798,812809,451550
181852,refseq,GCF_002872115.1,1676925,Paramormyrops kingsleyae,genomic,0,55174,37697302,56.10,57.17,...,189905,463292,403279,918378,335427,566434,388203,913712,520100,646555
181853,refseq,GCF_007399415.2,1825980,Gopherus evgoodei,genomic,0,49709,34253340,49.30,54.06,...,199608,300008,500034,677088,516372,422769,365846,548610,609307,517676


In [157]:
#change to percentage
df.iloc[:,12:76] = df.iloc[:,12:76].div(df["# Codons"].values, axis=0)
#add phylum data
df = df.merge(tax_df[["phylum", "Taxid"]], how="left")

In [161]:
#split into X and y data
X = df.iloc[:, 12:76]
y = df.iloc[:, 76]


0         28890
1         28890
2         28890
3         28890
4         28890
          ...  
193375     7711
193376     7711
193377     7711
193378     7711
193379     7711
Name: phylum, Length: 193380, dtype: int64